In [1]:
import pandas as pd
import requests
import pprint

# consulta a lista de deputados federais
response = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados')

# transforma em JSON
dados = response.json()

# transforma em DataFrame com Pandas
df_deputados = pd.DataFrame(dados['dados'])

# armazena os ids dos deputados
df_deputados_id = df_deputados['id']

#armazena os nomes dos deputados
df_deputados_nome = df_deputados['nome']

display(df_deputados_nome)

0       Abilio Brunini
1       Acácio Favacho
2          Adail Filho
3      Adilson Barroso
4         Adolfo Viana
            ...       
508           Zé Silva
509          Zé Trovão
510           Zé Vitor
511        Zeca Dirceu
512    Zezinho Barbary
Name: nome, Length: 513, dtype: object

In [23]:
contador = 0

# cria um DataFrame vazio
df = pd.DataFrame()

# consulta todas as despesas de todos os deputados federais
for requisicao in range(0,512):
    response_despesas = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados/{}/despesas?ano=2020&ano=2021&ano=2022&ano=2023&ordem=ASC&ordenarPor=ano'.format(df_deputados_id[contador]))
    dados = response_despesas.json()
    df_despesas = pd.DataFrame(dados['dados'])
    df_despesas['nome_deputado'] = df_deputados_nome[contador]
    df = pd.concat([df, df_despesas])
    contador = contador + 1

In [93]:
display(df)

,nome_deputado,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela
0,Acácio Favacho,2020.0,2.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7018213.0,Nota Fiscal Eletrônica,4.0,2020-02-18,1790,145.00,http://www.camara.leg.br/cota-parlamentar/nota...,BEL PONTO BORDADOS E CONFECCOES LTDA,00849026000109,145.00,0.0,,1678271.0,0.0
1,Acácio Favacho,2020.0,6.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7065252.0,Nota Fiscal Eletrônica,4.0,2020-06-16,545,229.00,http://www.camara.leg.br/cota-parlamentar/nota...,COMERCIAL DE SERVICOS UNIAO LTDA ME,05807200000174,229.00,0.0,,1702259.0,0.0
2,Acácio Favacho,2020.0,2.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7031733.0,Nota Fiscal,0.0,2020-03-03,417246,268.40,https://www.camara.leg.br/cota-parlamentar/doc...,EDVALDO FRANCISCO DE OLIVEIRA,32373812000179,268.40,0.0,,1684292.0,0.0
3,Acácio Favacho,2020.0,1.0,COMBUSTÍVEIS E LUBRIFICANTES.,6993481.0,Nota Fiscal Eletrônica,4.0,2020-01-06,479,3600.00,http://www.camara.leg.br/cota-parlamentar/nota...,A C CAMBRAIA LTDA,23068984000571,3600.00,0.0,,1666719.0,0.0
4,Acácio Favacho,2020.0,8.0,COMBUSTÍVEIS E LUBRIFICANTES.,7090596.0,Nota Fiscal Eletrônica,4.0,2020-08-03,128126,200.02,http://www.camara.leg.br/cota-parlamentar/nota...,AUTO POSTO 306 NORTE LTDA,22398818000147,200.02,0.0,,1715262.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Zeca Dirceu,2020.0,7.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7092392.0,Recibos/Outros,1.0,2020-07-31,sn,1300.00,https://www.camara.leg.br/cota-parlamentar/doc...,Antonio Peguim,04312880972,1300.00,0.0,,1716367.0,0.0
11,Zeca Dirceu,2020.0,8.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7092393.0,Recibos/Outros,1.0,2020-08-30,sn,1300.00,https://www.camara.leg.br/cota-parlamentar/doc...,Antonio Peguim,04312880972,1300.00,0.0,,1716365.0,0.0
12,Zeca Dirceu,2020.0,1.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7019946.0,Nota Fiscal,0.0,2020-01-24,118003178,235.28,https://www.camara.leg.br/cota-parlamentar/doc...,Copel Distribuição S.A.,04368898000106,235.28,0.0,,1679618.0,0.0
13,Zeca Dirceu,2020.0,2.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7044385.0,Nota Fiscal,0.0,2020-02-22,122568208,464.77,https://www.camara.leg.br/cota-parlamentar/doc...,Copel Distribuição S.A.,04368898000106,464.77,0.0,,1690671.0,0.0


In [22]:
# importando biblioteca para desenvolvimento dos gráficos
import plotly.express as px

# agrupa os dados por nome de deputado
df_despesas_por_deputados = df.groupby(['nome_deputado']).sum().reset_index()

# classifica os dados pela soma do valor líquido das despesas
df_despesas_por_top_5_deputados = df_despesas_por_deputados.sort_values('valorLiquido', ascending = False)
display(df_despesas_por_top_5_deputados)

# desenvolvimento do gráfico
fig = px.bar(df_despesas_por_top_5_deputados.head(10), x = 'valorLiquido', y = 'nome_deputado')
fig.update_layout(yaxis_autorange = 'reversed')
fig.show()

,nome_deputado,ano,mes,codDocumento,codTipoDocumento,valorDocumento,valorLiquido,valorGlosa,codLote,parcela
279,Vicentinho Júnior,30300.0,95.0,106031310.0,56.0,365000.00,365000.00,0.00,25557144.0,0.0
125,Gustinho Ribeiro,30300.0,86.0,106027407.0,0.0,337200.00,337200.00,0.00,25549769.0,0.0
138,Jhonatan de Jesus,30300.0,71.0,105770726.0,20.0,233102.90,233102.90,0.00,25421725.0,0.0
263,Silas Câmara,30300.0,76.0,105992292.0,12.0,185006.39,185006.39,0.00,25537126.0,0.0
78,Domingos Neto,30300.0,93.0,106078064.0,0.0,180000.00,173870.99,6129.01,25579635.0,0.0
...,...,...,...,...,...,...,...,...,...,...
270,Tabata Amaral,30300.0,102.0,106268692.0,2.0,994.95,991.28,3.67,25677331.0,0.0
204,Miguel Lombardi,30300.0,89.0,106194888.0,0.0,680.23,670.38,9.85,25635456.0,0.0
86,Eduardo Bolsonaro,30300.0,64.0,105631499.0,0.0,582.61,582.61,0.00,25353571.0,0.0
14,Alfredinho,2023.0,1.0,0.0,1.0,467.60,467.60,0.00,0.0,0.0
